<hr style="height: 1px;">
<i>This notebook was authored by the 8.316 Course Team, Copyright 2023 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 7: Correlations</h1>


<a name='section_7_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_1">L7.1 Understanding Best Fit (Revisited)</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_1">L7.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_2">L7.2 Minimizing on a Surface (1D Scan)</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_2">L7.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_3">L7.3 Minimizing on a Surface (2D Scan)</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_3">L7.3 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_4">L7.4 Correlations Between Fit Parameters: Part 1</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_4">L7.4 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_5">L7.5 Correlations Between Fit Parameters: Part 2</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_5">L7.5 Exercises</a></td>
    </tr>
</table>



<h3>Learning Objectives</h3>

In this lecture, we wil exploret the nature of correlated variables and how they impact our interpretation of results. Furthermore, we will understand how to go away from correlations to see how we can reduce their impact. Additionally, we will start to scan the parameters of our fit beyond just finding the minimum. By scanning, we start to learn the subtleties of each of these setups, and we start to get deep insight into the nature of what we are minimizing and how to interpret it. 


<h3>Importing Libraries</h3>

Before beginning, run the cells below to import the relevant libraries for this notebook. If you're using a Colab cloud runtime, also run the cell labeled "for Colab users."


In [ ]:
#>>>RUN: L7.0-runcell01

import numpy as np
import matplotlib.pyplot as plt


In [ ]:
#>>>RUN: L7.0-runcell02

# for Colab users
!pip install lmfit

# Get data and download to root of your Google Drive
from google.colab import drive
drive.mount('/content/drive')

# If you're not comfortable giving permissions for this command, download the file from the URL, 
# then upload to your Google Drive root folder.
!wget -P /content/drive/MyDrive/ https://raw.githubusercontent.com/mitx-8s50/data/main/L07/events_a8_1space.dat

In [ ]:
#>>>RUN: L7.0-runcell02 (ANOTHER WAY?)

# for Colab users
!pip install lmfit

# Downloading data for your runtime
!cd /
!wget https://raw.githubusercontent.com/mitx-8s50/data/main/L07/events_a8_1space.dat

<h3>Setting Default Figure Parameters</h3>

The following code cell sets default values for figure parameters.

In [ ]:
#>>>RUN: L7.0-runcell03

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title


<a name='section_7_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.3 Fit to Full Cosmological Model</h2>     

| [Top](#section_7_0) | [Previous Section](#section_7_2) | [Exercises](#exercises_7_3) |


<h3>A more sophisticated fit</h3>

Now that we have gone on an excursion to understand properties of fits, let's go ahead and analyze our supernovae data, and try to pull in all of the information that we can. Let's first look at our linear fit.  One sec, while we load it all:


In [ ]:
#>>>RUN: L6.6-runcell01
import math
import numpy as np
import csv
import matplotlib.pyplot as plt
from scipy import stats

#Let's try to understand how good the fits we made in last Lesson are, let's load the supernova data again
label='data/sn_z_mu_dmu_plow_union2.1.txt'

def distanceconv(iMu):
    power=iMu/5+1
    return 10**power

def distanceconverr(iMu,iMuErr):
    power=iMu/5+1
    const=math.log(10)/5.
    return const*(10**power)*iMuErr

def load(iLabel,iMaxZ=0.1):
    redshift=np.array([])
    distance=np.array([])
    distance_err=np.array([])
    with open(iLabel,'r') as csvfile:
        plots = csv.reader(csvfile, delimiter='\t')
        for row in plots:
            if float(row[1]) > iMaxZ:
                continue
            redshift = np.append(redshift,float(row[1]))
            distance = np.append(distance,distanceconv(float(row[2])))
            distance_err = np.append(distance_err,distanceconverr(float(row[2]),float(row[3])))
    return redshift,distance,distance_err  
        
redshift,distance,distance_err=load(label,10000)
plt.errorbar(redshift,distance,yerr=distance_err,marker='.',linestyle = 'None', color = 'black')
plt.xlabel("z(redshift)")
plt.ylabel("distance(pc)")
plt.show()



In [ ]:
#>>>RUN: L6.7-runcell02

#We are not going to plot the fit first, let's just use our barrage of statistics to check if its ok
def sqrtTerm(z,Om):
    pVal=Om*(1+z)**3+(1.-Om)
    return np.sqrt(pVal)

def lumidistance(z,h0,Om):
    integral=0
    nint=100
    for i0 in range(nint):
        zp=z*float(i0)/100.
        dz=z/float(nint)
        pVal=1./(1e-5+sqrtTerm(zp,Om))###<=== note the 1e-5 hack
        integral += pVal*dz
    d=(1.+z)*integral*(1e6*3e5/h0)
    return d

print("test Lumi",lumidistance(1,70,0.3))


<a name='exercises_6_6'></a>     

| [Top](#section_6_0) | [Restart Section](#section_6_6) | [Next Section](#section_6_7) |


Now let's go and run our fit function that included the parameters of the universe. As a first pass, let's just pipe this through lmfit and see if it works. 

In [ ]:
#>>>RUN: L6.8-runcell01

import lmfit

model  = lmfit.Model(lumidistance)
p = model.make_params(h0=60,Om=0.2)
result = model.fit(data=distance, params=p, z=redshift, weights=1./distance_err)
lmfit.report_fit(result)
plt.figure()
result.plot()


Now, since we are experts at fitting, let's go ahead and compute the log likelihood . We are going to minimize the (negative of) loglikelihood with `scipy.optimize` in order to do the fit.

In [ ]:
#>>>RUN: L6.8-runcell02
from scipy import optimize as opt 

def chi2(x):
    xtest=lumidistance(redshift,x[0],x[1])
    val=(distance-xtest)/distance_err
    return np.sum(val**2)
    
def residuals(x):
    residuals=np.array([])
    for i0 in range(len(redshift)):
        pResid=lumidistance(redshift[i0],sol.x[0],sol.x[1])-distance[i0]
        residuals = np.append(residuals,pResid/distance_err[i0])
    return residuals


x0 = np.array([70.,0.3])
ps = [x0]
bnds = ((0, 1000), (0, 1.0))
sol=opt.minimize(chi2, x0,bounds=bnds)#, tol=1e-6)
print(sol)
residuals=residuals(sol.x)
chi2t=np.sum(residuals**2)
print("Total chi2:",chi2t,"NDOF",len(residuals)-2)
print("Normalized chi2:",chi2t/(len(residuals)-2))
print("Probability of chi2:",1-stats.chi2.cdf(chi2t,(len(residuals)-2)))

#Let's plot it for good measure too
x = np.linspace(0,len(residuals)*2)
chi2d=stats.chi2.pdf(x,len(residuals-2)) # 40 bins
plt.plot(x,chi2d,label='chi2')
plt.axvline(chi2t, c='red')
plt.legend(loc='lower right')
plt.show()

#xvals = np.linspace(0.,20.,100)
#yvals = []
#for pX in xvals:
#    yvals.append(chi2([70,pX]))
#plt.plot(xvals,yvals)
#plt.show()

What can we say about this fit, is there something off?

Let's plot the residuals and the fit function and also scan the likelihood for our parameter uncertainties. There is one thing off, can you figure it out?


In [ ]:
#>>>RUN: L6.8-runcell03

#Plot it against the data
xvals = np.linspace(0,1.4,100)
yvals = []
for pX in xvals:
    yvals.append(lumidistance(pX,sol.x[0],sol.x[1]))

plt.errorbar(redshift,distance,yerr=distance_err,marker='.',linestyle = 'None', color = 'black')
plt.plot(xvals,yvals)
plt.show()

#Histogram the residuals
y0, bin_edges = np.histogram(residuals, bins=30)
bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])
norm0=len(residuals)*(bin_edges[-1]-bin_edges[0])/30.
plt.errorbar(bin_centers,y0/norm0,yerr=y0**0.5/norm0,marker='.',drawstyle = 'steps-mid')
k=np.arange(bin_edges[0],bin_edges[-1],0.05)
normal=stats.norm.pdf(k,0,1)
plt.plot(k,normal,'o-')
plt.show()

x = np.linspace(len(residuals)*0.5,len(residuals)*1.5)
chi2d=stats.chi2.pdf(x,len(residuals-2)) # 40 bins
plt.plot(x,chi2d,label='chi2')
plt.axvline(chi2t, c='red')
plt.legend(loc='lower right')
plt.show()




<a name='exercises_6_8'></a>   

| [Top](#section_6_0) | [Restart Section](#section_6_8) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-7.6.1</span>

So given the Friedmann equations, we can add back the curvature term:

$$ d(z) = ct^{\prime} = (1+z)ct = (1+z)\frac{c}{h_{0}}\int_{0}^{z} \frac{dz^{\prime}}{\sqrt{\Omega_{M}\left(1+z^{\prime}\right)^{3} + \Omega_{\kappa}\left(1+z^{\prime}\right)^{2}+ 1-\Omega_{M}-\Omega_{\kappa}}}$$

Adjust the `lumidistance()` function to fit for curvature. What is the value of $\Omega_{\kappa}$ if we perform a new fit to the data? What do you think this this is implying? 

Enter your answer as a number with precision 1e-2.


In [ ]:
#>>>EXERCISE
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def hubble_curve(z,Om,OmK):
    return #YOUR CODE HERE

def lumidistance_curve(z,h0,Om,OmK):
    return #YOUR CODE HERE


model  = lmfit.Model(lumidistance_curve)
p = model.make_params(h0=70,Om=0.2,OmK=0.0)
result = model.fit(data=distance, params=p, z=redshift, weights=1./distance_err)
lmfit.report_fit(result)
result.plot();


In [ ]:
#>>>SOLUTION
def hubble_curve(z,Om,OmK):
    pVal=Om*(1+z)**3+OmK*(1+z)**2+(1.-Om-OmK)
    return np.sqrt(pVal)

def lumidistance_curve(z,h0,Om,OmK):
    integral=0
    nint=100
    for i0 in range(nint):
        zp=z*float(i0)/100.
        dz=z/float(nint)
        pVal=1./(1e-5+hubble_curve(zp,Om,OmK))
        integral += pVal*dz
    d=(1.+z)*integral*(1e6*3e5/h0)
    return d


model  = lmfit.Model(lumidistance_curve)
p = model.make_params(h0=70,Om=0.2,OmK=0.0)
result = model.fit(data=distance, params=p, z=redshift, weights=1./distance_err)
lmfit.report_fit(result)
result.plot();

<div style="border:1.5px; border-style:solid; padding: 0.5em; border-color: #90409C; color: #90409C;">

**SOLUTION:**

<pre>
-0.19 +/- 0.40
</pre>
        
**EXPLANATION:**
    
We find the curvature is consistent with 0, with a slight bias towards a negative curvature value. Interestingly, we also find that the Hubble's constant doesn't change as much, but the matter changes some more. However, the overall scale of change is relatively small. Interestingly, the chi2 value is almost exactly the same, implying that adding these parameters does not really change the overall performance. 
    
</div>


<a name='section_7_7'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.7 Understanding Best Fit (Revisited)</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_6) | [Exercises](#exercises_7_7) | [Next Section](#section_7_8) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.1x+3T2022/block-v1:MITxT+8.S50.1x+3T2022+type@sequential+block@seq_LS7/block-v1:MITxT+8.S50.1x+3T2022+type@vertical+block@vert_LS7_vid1" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Slides</h3>

Run the code below to view the slides for this section, which are discussed in the related video. You can also open the slides in a separate window <a href="https://mitx-8s50.github.io/slides/L07/slides_L07_01.html" target="_blank">HERE</a>.

In [ ]:
#>>>RUN: L7.1-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L07/slides_L07_01.html', width=970, height=550)

<h3>Slides</h3>

View the slides for this section below, which are discussed in the related video. You can also open the slides in a separate window <a href="https://mitx-8s50.github.io/slides/L07/slides_L07_01.html" target="_blank">HERE</a>.

<p align="center">
<iframe src="https://mitx-8s50.github.io/slides/L07/slides_L07_01.html" width="900", height="550" frameBorder="0"/>
</p>

<h3>Uncertainty on more than one parameter</h3>

At the end of last week, we started to fit more than one parameter. That is, we minimized some error metric for more than one parameter. Let's take a closer look. As before, we'll fit the function 

$$
\begin{equation}
 f(x,a,b) = a + b \sin(x)
\end{equation}
$$

to the Auger data, i.e. finding optimal choices of parameters $a$ and $b$ given some set of sample observations $y_i$ and their sample uncertainties $\sigma_i$. One way to choose optimal parameter values is by optimizing our likelihood over both $a$ and $b$. As a reminder, likelihood defines  how probable our observed data is as a function of parameter values and a hypothesized form. At extrema, we'd expect our likelihood is maximized or 

$$
\begin{equation}
 \frac{\partial \mathcal{L}}{\partial a} = 0 \\
 \frac{\partial \mathcal{L}}{\partial b} = 0 \\
\end{equation}
$$

As we saw in L5.7, maxima of the likelihood function correspond to minima of the relevant $\chi^2$ statistic of the form

$$
\begin{equation}
  \chi^{2}(x|a,b) = \sum_{i=1}^{N} \frac{(y_{i}-f(x_{i},a,b))^2}{\sigma_i^2}
\end{equation}
$$

At the extrema,

$$
\begin{eqnarray}
\frac{\partial \chi^{2}(x|a,b) }{\partial a} = 0 \\
\frac{\partial \chi^{2}(x|a,b) }{\partial b} = 0
\end{eqnarray}
$$

In a little bit, we're going to use this to manually write code to fit the function using the generic optimizer in `scipy`. But first, let's review how things look using `lmfit`. 

In [ ]:
#>>>RUN: L7.1-runcell01

############ This is all code from a previous lesson
import numpy as np
import csv
import math
import lmfit
import matplotlib.pyplot as plt
from scipy import optimize as opt

def rad(iTheta):
    return iTheta/180. * math.pi

def rad1(iTheta):
    return iTheta/180. * math.pi-math.pi

def load(label):
    dec=np.array([])
    ra=np.array([])
    az=np.array([])
    with open(label,'r') as csvfile:
        plots = csv.reader(csvfile,delimiter=' ')
        for pRow in plots:
            if '#' in pRow[0] or pRow[0]=='':
                continue
            dec = np.append(dec,rad(float(pRow[2])))
            ra  = np.append(ra,rad1(float(pRow[3])))
            az  = np.append(az,rad(float(pRow[4])))
    return dec,ra,az

def prephist(iRA):
    y0, bin_edges = np.histogram(iRA, bins=30)
    x0 = 0.5*(bin_edges[1:] + bin_edges[:-1])
    y0 = y0.astype('float')
    return x0,y0,1./(y0**0.5)

label8='data/events_a8_1space.dat'

# Uncomment if using cloud runtime (which one?)
#label8='/content/drive/MyDrive/events_a8_1space.dat'
#label8='/content/events_a8_1space.dat'

dec,ra8,az=load(label8)
xhist,yhist,xweights=prephist(ra8)


########## Tlast fit code

def fnew(x,a,b):
    return a+b*np.sin(x)

model  = lmfit.Model(fnew)
p = model.make_params(a=1000,b=10)
result = model.fit(data=yhist,x=xhist, params=p, weights=xweights)
lmfit.report_fit(result)
plt.figure()
result.plot()
plt.show()


<h3> What is the $\chi^{2}$-value from a p-value? </h3>

So what is the importance of this fit? This is just equal to a $\chi^{2}$ distribution, as we ahve seen before. Before we delve into the details of the importance of the fit, lets just remind ourselves how to compute the p-value from a gaussian and the $\chi^{2}$ value from the p-value or a sigma value assuming a Gaussian. 


In [ ]:
#>>>RUN: L7.1-runcell02

import scipy.stats as stats

sigma = 1
#print(stats.norm.cdf(sigma)-stats.norm.cdf(-sigma))

def pval(iVal):
    return stats.norm.cdf(iVal)-stats.norm.cdf(-iVal)

def chi2Val(iGausSigma,iNDOF):
    val=stats.chi2.ppf(pval(iGausSigma),iNDOF)
    return val

print(chi2Val(2,28))

<a name='exercises_7_1'></a>     

| [Top](#section_7_0) | [Restart Section](#section_7_1) | [Next Section](#section_7_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-7.7.1</span>

Compute the Chi-square value for 2 sigma and 2 degrees of freedom? Enter your answer as number with precision 1e-2. 


In [ ]:
#>>>EXERCISE
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def exp_func(x):
    return 0


In [ ]:
#>>>SOLUTION

import scipy.stats as stats

def pval(iVal):
    return stats.norm.cdf(iVal)-stats.norm.cdf(-iVal)

def chi2Val(iGausSigma,iNDOF):
    val=stats.chi2.ppf(pval(iGausSigma),iNDOF)
    return val

print(chi2Val(2,2))


<div style="border:1.5px; border-style:solid; padding: 0.5em; border-color: #90409C; color: #90409C;">

**SOLUTION:**

$Χ^{2} = 6.18$

        
**EXPLANATION:**
    
Use the same function as explained in L7.1 lecture video. 
    
</div>


<a name='section_7_8'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.8 Minimizing on a Surface (1D Scan)</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_7) | [Exercises](#exercises_7_8) | [Next Section](#section_7_9) |


<h3>Overview</h3>

Now, what we are going to do is re-run the fit. Here we'll go through the $\chi^2$ minimization process semi manually, using `scipy.opt`. The code is explained in the lecture.

Strategically, what we are trying to do is write the $\chi^{2}$ value and use our normal minimizer tools to find the minimum parameters. 


In [ ]:
#>>>RUN: L7.2-runcell01

#This is the new code
def chi2(iX): #iX is an array iX[0]=a and iX[1]=b
    '''Note that this function depends on xhist and yhist being defined already.
    This is bad practice in general, but we do it here for convenience.
    After all, this code isn't reused elsewhere.'''
    assert len(iX) == 2
    lTot=0
    for val in range(len(yhist)):
        xtest=fnew(xhist[val],iX[0],iX[1])
        lTot += (1./(1e-5+xtest))*(yhist[val]-xtest)**2
    return lTot

#First we minimize
x0 = np.array([1000,10]) # initial conditions
ps = [x0]
sol=opt.minimize(chi2, x0)
print(sol)

Now, given that we have found the minimum, lets look at how the $\chi^{2}$ value changes from the minimum. The important thing is to remember that from [Wilk's Thereom](https://en.wikipedia.org/wiki/Wilks%27_theorem) the uncertainty of a parameter can be obtained by tkaing $\Delta \chi^{2}$ from the minimum of a value of 1. Lets go ahead and plot the minimum and look at what $\Delta \chi^{2}=1$ looks like. 

In [ ]:
#>>>RUN: L7.2-runcell02

#Scan near the minimum of each value
x = np.linspace(sol.x[0]*0.99,sol.x[0]*1.01, 100)
y = np.linspace(sol.x[1]*0.5,sol.x[1]*1.5, 100)

#Now lets fix one parameter at the minimum, and profile the other
plt.plot(x, chi2([x,sol.x[1]]),label='chi2');
plt.axhline(sol.fun+1, c='red')
plt.xlabel("a-value")
plt.ylabel("$\chi^{2}$")
plt.show()

#Now for the other parameter
plt.plot(y, chi2([sol.x[0],y]),label='chi2');
plt.axhline(sol.fun+1, c='red')
plt.xlabel("b-value")
plt.ylabel("$\chi^{2}$")
plt.show()

Where the red and blue lines cross correspnods to the one $\sigma$ fluctation up and down from the minimum. As a result, we can code up this algorithm and compute the best fit and the uncertainty on the best fit parameter. Let's go ahead and code this up. 

In [ ]:
######## Now lets use a numerical solver to find the points at which a function crosses zero (root solver)
def chi2minX(xval, delta_chi2=1):
    val=chi2([xval,sol.x[1]])
    minval=chi2(sol.x) + delta_chi2
    return val-minval

def chi2minY(yval, delta_chi2=1):
    val=chi2([sol.x[0],yval])
    minval=chi2(sol.x) + delta_chi2
    return val-minval

def chi2uncX(sol):
    solX1=opt.root_scalar(chi2minX,bracket=[sol.x[0], sol.x[0]*1.02],method='brentq')
    solX2=opt.root_scalar(chi2minX,bracket=[sol.x[0]*0.98, sol.x[0]],method='brentq')
    print("a:",sol.x[0],"+/-",abs(solX2.root-solX1.root)/2.)
    print("Reminder the Poisson uncertainty would be:",math.sqrt(np.mean(yhist)/(len(xhist))))
    return solX1, solX2

def chi2uncY(sol):
    solY1=opt.root_scalar(chi2minY,bracket=[sol.x[1],    sol.x[1]*1.2],method='brentq')
    solY2=opt.root_scalar(chi2minY,bracket=[sol.x[1]*0.8, sol.x[1]],method='brentq')
    print("b:",sol.x[1],"+/-",abs(solY2.root-solY1.root)/2.)
    return solY1, solY2

solX1, solX2 = chi2uncX(sol)
solY1, solY2 = chi2uncY(sol)

So what have we done? 

We've used the optimizer to minimize the $\chi^{2}$ values in 2D (a,b), and we have obtained an uncertainty by looking at $\Delta \chi^{2}$. You can see that our semi manual manipulation got us to the same parameter estimates as the `lmfit` example. Additionally, by looking at slices of $\chi^{2}$ in each of the parameters we obtained the same uncertainties. 



<a name='exercises_7_2'></a>     

| [Top](#section_7_0) | [Restart Section](#section_7_2) | [Next Section](#section_7_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-7.8.1</span>

What are the 2$\sigma$ bounds (that is, 95.45% confidence interval values) of $a$ and $b$ in the fit above? Enter your answer as a list of numbers number with precision 1 (the nearest whole number): `[a_lower, a_upper, b_lower, b_upper]`

In [ ]:
#>>>EXERCISE

#For a given parameter, assume everything else is determined, so we're
# working with 1 degree of freedom.
# First, we want to know the chi-square value at which 2 standard deivations (95.45% of the probability)
# in the chi-square distribution is for values less extreme than that value.
ndof=1
pval=0.9545
val = stats.chi2.ppf(pval,ndof)
print("Delta Chi-square:", val)

def minXfunc(x):
    return chi2minX(x, delta_chi2=val)

def minYfunc(x):
    return chi2minY(x, delta_chi2=val)

def chi2unc(sol, sol_index, min_func, unc_prop_guess):
 #insert code here
 return valmin,valmax

a1, a2 = chi2unc(sol, 0, minXfunc, 0.08)
b1, b2 = chi2unc(sol, 1, minYfunc, 0.4)

print(f"a bounds: [{a1}, {a2}]")
print(f"b bounds: [{b1}, {b2}]")


In [ ]:
#>>>SOLUTION

from scipy.optimize.zeros import RootResults
from scipy.optimize.optimize import OptimizeResult
from typing import Callable, Tuple

# For a given parameter, assume everything else is determined, so we're
# working with 1 degree of freedom.
# First, we want to know the chi-square value at which 95.45% of the probability
# in the chi-square distribution is for values less extreme than that value.
# We do this as before:
ndof=1
pval=0.9545
val = stats.chi2.ppf(pval,ndof)
print("Delta Chi-square:", val)

# Now, we independently perturb a or b from their minimized chi-square values
# until this Delta chi-square value is achieved, just like above.

def chi2unc(sol, sol_index, min_func, unc_prop_guess):
    '''Does what chi2uncX and chi2uncY above do, with unc_prop_guess defining the
    proportional size of search space (e.g. 0.2 in chi2uncY, or 0.02 in chi2uncX)
    However, returns the parameter values where the delta chi2 is reached'''
    solX1=opt.root_scalar(min_func, bracket=[sol.x[sol_index], sol.x[sol_index]*(1+unc_prop_guess)], method='brentq')
    solX2=opt.root_scalar(min_func, bracket=[sol.x[sol_index]*(1-unc_prop_guess), sol.x[sol_index]], method='brentq')
    return solX1.root, solX2.root

def minXfunc(x):
    return chi2minX(x, delta_chi2=val)

def minYfunc(x):
    return chi2minY(x, delta_chi2=val)

# Here we set our delta_chi2 to the value in question, and guess how far away
# the values of a and b will be to set unc_prop_guess (if not big enough, error)
# then let the optimizer find the corresponding a and b uncertainties
a1, a2 = chi2unc(sol, 0, minXfunc, 0.08)
b1, b2 = chi2unc(sol, 1, minYfunc, 0.4)

print(f"a bounds: [{a1}, {a2}]")
print(f"b bounds: [{b1}, {b2}]")



<div style="border:1.5px; border-style:solid; padding: 0.5em; border-color: #90409C; color: #90409C;">


**SOLUTION:**

<pre>
a in roughly [1061, 1085] and b in roughly [-67, -33]
</pre>
        
**EXPLANATION:**
    
We've assumed all but one parameter is fully determined, so we have 1 degree of freedom. For sake of explanation, assume it's $a$.

We're looking for 2-sigma (95.45%) confidence interval bounds, that is, the values of $a$ for which $\Delta \chi^2$ is such that in a 1 DoF $\chi^2$ distribution, 95% of $\chi^2$ values are less extreme.
From the inverse of the CDF, we find this $\Delta \chi^2$ is 4.

Doing root finding to figure out how far we need to move $a$ or $b$ to achieve this increase in $\chi^2$, we find the values we're looking for.
    
</div>


<a name='section_7_9'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.9 Minimizing on a Surface (2D Scan)</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_8) | [Exercises](#exercises_7_9) | [Next Section](#section_7_10) |


Now given our minimization worked in 1D for both variables $a$ and $b$, what about if we scan both variables simultaneously and then look at how the minimum behaves. To do this, we can start by first scaning around the best fit values of $a$ and $b$ and just plotting the value of our $\chi^{2}$ minimization. From this space, we can start to relate this back to the 2D version of Wilk's theorem in the slides below. 

<h3>Slides</h3>

Run the code below to view the slides for this section, which are discussed in the related video. You can also open the slides in a separate window <a href="https://mitx-8s50.github.io/slides/L07/slides_L07_03.html" target="_blank">HERE</a>.

In [ ]:
#>>>RUN: L7.3-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L07/slides_L07_03.html', width=970, height=550)

<h3>Slides</h3>

View the slides for this section below, which are discussed in the related video. You can also open the slides in a separate window <a href="https://mitx-8s50.github.io/slides/L07/slides_L07_03.html" target="_blank">HERE</a>.

<p align="center">
<iframe src="https://mitx-8s50.github.io/slides/L07/slides_L07_03.html" width="900", height="550" frameBorder="0"/>
</p>

To really visualize the whole thing lets make one more plot: how the $\chi^2$ value depends on both parameters. We can do this using the `meshgrid` function and plotting it. In addition, we will add some color full lines for a $\Delta \chi^{2}$ given by various values, we will revisit what these values are in a little bit.   

In [ ]:
#>>>RUN: L7.3-runcell01

#define the 2D X and Y grid
x = np.linspace(sol.x[0]*0.99,sol.x[0]*1.01, 100) #grid in a
y = np.linspace(sol.x[1]*0.5,sol.x[1]*1.5, 100)#grid in b
X, Y = np.meshgrid(x, y) #2d grid
#print(x[0],y[0],X[1,0],Y[1,0])

# For z coordinate, evaluate chi2 at each x,y point in the grid.
# note understanding this one-liner itself isn't too important
Z = np.array([chi2([x,y]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)


#and plot
def plotColorsAndContours(X,Y,Z):
    fig, ax = plt.subplots(1, 1)
    c = ax.pcolor(X,Y,Z,cmap='RdBu')
    cb=fig.colorbar(c, ax=ax)
    plt.xlabel("a")
    plt.ylabel("b")
    cb.set_label("$\chi^{2}$")
    #Now lets plot the contours of Delta chi^2
    levels = [0.1,1,2.3,4,9, 16, 25, 36, 49, 64, 81, 100]
    for i0 in range(len(levels)):
        levels[i0] = levels[i0]+sol.fun
    c = plt.contour(X, Y, Z, levels,colors=['red', 'blue', 'yellow','green'])
    #plt.show()
    
plotColorsAndContours(X,Y,Z)

On the above we've plotted level curves representing different $\chi^2$ increases from the minimum. In the last section, we obtained uncertainties for our fit parameters by allowing just 1 varaible to vary, then observing the $\Delta\chi^2$. We may ask, what is the uncertainty profile when we are letting 2 float simultaneously?

Lets go back to our lecture slides' where we have the Taylor expansion result that led to the relation of Wilk's theorem. Now, we can write the expansion in terms of all variables $\vec{\theta}$:  

$$
\begin{equation}
\chi^{2}(x_{i},\vec{\theta})=\chi^{2}_{min}(x_{i},\vec{\theta})+\frac{1}{2}(\theta_{i}-\theta_{j})^{T}\frac{\partial^{2}}{\partial \theta_{i}\theta_{0}}\chi^{2}_{min}(x_{i},\vec{\theta}_{0})(\theta_{j}-\theta_{0})
\end{equation}
$$

We can write this out in 2D as:

$$
\begin{equation}
\chi^{2}(x,\vec{\theta})=\chi_{min}^{2}(x,\vec{\theta})+\frac{1}{2}\left(\begin{array}{cc}
\theta_{a}-\theta_{a-min} & \theta_{b}-\theta_{b-min}\end{array}\right)\left(\begin{array}{cc}
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}^{2}} & \frac{\partial^{2}\chi^{2}}{\partial\theta_{a}\partial\theta_{b}}\\
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}\partial\theta_{b}} & \frac{\partial^{2}\chi^{2}}{\partial\theta_{b}^{2}}
\end{array}\right)\left(\begin{array}{c}
\theta_{a}-\theta_{a-min}\\
\theta_{b}-\theta_{b-min}
\end{array}\right)
\end{equation}
$$

In the case where $\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}\partial\theta_{b}}\approx0$ we can simplify this approximation a lot.

$$
\begin{equation}
\chi^{2}(x,\vec{\theta})=\chi_{min}^{2}(x,\vec{\theta})+\frac{1}{2}\left(\begin{array}{cc}
\Delta\theta_{a} & \Delta\theta_{b}\end{array}\right)\left(\begin{array}{cc}
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}^{2}} & 0\\
0 & \frac{\partial^{2}\chi^{2}}{\partial\theta_{b}^{2}}
\end{array}\right)\left(\begin{array}{c}
\Delta\theta_{a}\\
\Delta\theta_{b}
\end{array}\right)
\end{equation}
$$

This all becomes a 2D quadratic equation

$$
\begin{align*}
\chi^{2}(x,\vec{\theta}) & =\chi_{min}^{2}(x,\vec{\theta})+\frac{1}{2}\left(\begin{array}{cc}
\Delta\theta_{a}^{2}\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}^{2}}+ & \Delta\theta_{b}^{2}\frac{\partial^{2}\chi^{2}}{\partial\theta_{b}^{2}}\end{array}\right)\\
 & =\chi_{min}^{2}(x,\vec{\theta})+\left(\begin{array}{cc}
\frac{\Delta\theta_{a}^{2}}{\sigma_{\theta_{a}}^{2}}+ & \frac{\Delta\theta_{b}^{2}}{\sigma_{\theta_{b}}^{2}}\end{array}\right)
\end{align*}
$$

I want to point out, we are now profiling two parameters at once in this 2D plot, which means the contribution to $\chi^{2}$ involves 2 degrees of freedom. You can see this from the above equation, since its the sum of 2 gaussian distributed variables with width $1$ and mean $0$. The 1 $\sigma$ confidence interval for 2 degrees of freedom is computed by taking $\Delta \chi^2(x,\nu=2)=x~\mathrm{where~}\mathrm{cdf}\left(\chi^{2}(x,2)=0.683\right)\approx2.3$. This is the yellow contour. Contrast this with the case of 1 $\sigma$ confidence on 1 degree of freedom, where $\Delta\chi^2 = 1$. 

Now, we can go one step further, and note that the above formula  $ a x^2 + b y^2 = c$ is just the form of an ellipse with a specific major and minor axis. As a result, we can define a 3D funciton given by 

$$ f(x,y) = \left(\frac{x-\bar{a}}{\sigma_{a}}\right)^{2} + \left(\frac{y-\bar{b}}{\sigma_{b}}\right)^{2} $$

Anyway, let's plot the ellipses and compare it to our minimum contours. 


In [ ]:
#>>>RUN: L7.3-runcell02

#Lets plot the uncertainties  from hess_inv
print(np.sqrt(2*sol.hess_inv))
#the diagonals are approximately the errors

#Make a the expression in the above equation x and x0 are 2 vectors
def quadratic2D(x,x0,sigma0):
    lVals=x-x0
    lVals=(lVals**2)/(sigma0)/sigma0
    return np.sum(lVals)

plotColorsAndContours(X,Y,Z)

#Now plot the ellipse in 3D
def plotEllipse(sigx,sigy):
    levels = [0.1,1,2.3,4,9, 16, 25, 36, 49, 64, 81, 100]
    ZQ = np.array([quadratic2D([x,y],sol.x,[sigx,sigy]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)
    c = plt.contour(X, Y, ZQ, levels,colors=['red', 'blue', 'yellow','green'],linestyles='dashed')

sigx=(solX2.root-solX1.root)/2.
sigy=(solY2.root-solY1.root)/2.
plotEllipse(sigx,sigy)
plt.show()

Remarkably, our space, give by the formula above, yields the solid lines, which matches very closely to the dashed contours obtained by just writing the $\chi^{2}$ value. This is a profound statement, it really is. 

Alright, since this gives us the same yellow line for our 2-variable 1$\sigma$ confidence ellipse. If you look close you do see a difference. This makes us beg the question of what happens when $\frac{\partial^{2}\chi^{2}}{\partial \theta_{a}\theta_{b}}\neq0$. We'll see soon, when we look at correlations on fitting a different set of parameters.

<a name='exercises_7_3'></a>     

| [Top](#section_7_0) | [Restart Section](#section_7_3) | [Next Section](#section_7_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-7.9.1</span>

When we allow 2 parameters to vary, for a given confidence level (e.g. 1$\sigma$) we end up with a confidence ellipse containing parameter values outside the confidence intervals on the individual parameters alone. Compute the 1$\sigma$ (68.27%) confidence interval bounds for the parameter $a$, based on this ellipse from floating both $a$ and $b$. Enter your answer as a list of number with precision 1 (nearest whole number): `[a_lower, a_upper]`

Why do we typically quote the uncertainty from the 1D variation?


In [ ]:
#>>>EXERCISE
# We can follow the same procedure for finding the confidence interval based on 1D
# but for 1-sigma we now use 2.3 
# Use the code from the solution to Ex. 7.2.1,
ndof=insert value here
pval=0.6827#1 sigma p-value
val = stats.chi2.ppf(pval,ndof) # 2 DoF this time!
print("Delta Chi-square:", val)

def minfunc(x):
    return chi2minX(x, delta_chi2=val)

print(f"a bounds: [{a1}, {a2}]")


In [ ]:
#>>>SOLUTION

# We can follow the same procedure for finding the confidence interval based on 1D
# but for 1-sigma we now use 2.3 instead of 1 for delta chi^2, since we have 2 DoF.
# Using the code from the solution to Ex. 7.2.1,
val = stats.chi2.ppf(0.6827,2) # 2 DoF this time!
print("Delta Chi-square:", val)

def minfunc(x):
    return chi2minX(x, delta_chi2=val)

a1, a2 = chi2unc(sol, 0, minfunc, 0.2)

print(f"a bounds: [{a1}, {a2}]")


<div style="border:1.5px; border-style:solid; padding: 0.5em; border-color: #90409C; color: #90409C;">

**SOLUTION:**

<pre>
a in roughly [1064, 1082]
</pre>
        
**EXPLANATION:**
    
Here we find the estimate on $a$ to have uncertainty around $\pm 9$, as opposed to around $\pm 6$ when we had only $a$ vary. For consistency, we usually quote the uncertainty on just one variable at a time.
    
</div>


<a name='section_7_10'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.10 Correlations Between Fit Parameters: Part 1</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_9) | [Exercises](#exercises_7_10) | [Next Section](#section_7_11) |


Ok, so now lets try to understand what happens when our off diagonal terms are off. Let's go back to Wilk's theorem and look at the Hessian for the $\chi^{2}$, but now with non-zero off-diagonal parameters. 

<h3>Slides</h3>

Run the code below to view the slides for this section, which are discussed in the related video. You can also open the slides in a separate window <a href="https://mitx-8s50.github.io/slides/L07/slides_L07_04.html" target="_blank">HERE</a>.

In [ ]:
#>>>RUN: L7.4-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L07/slides_L07_04.html', width=970, height=550)

<h3>Slides</h3>

View the slides for this section below, which are discussed in the related video. You can also open the slides in a separate window <a href="https://mitx-8s50.github.io/slides/L07/slides_L07_04.html" target="_blank">HERE</a>.

<p align="center">
<iframe src="https://mitx-8s50.github.io/slides/L07/slides_L07_04.html" width="900", height="550" frameBorder="0"/>
</p>

To understand the point of this, what we are going to do is make the 2D contour plot like the one above with the same contours as before, but now we will use a new function, that is just a reparametrization of the old one. This function will be

$$
\begin{equation}
 f(x) = a x + b (1-x)
\end{equation}
$$

This means that the values of $a$ and $b$ now have different meaning, but this is just a linear fit like before and it should give the same overall $\chi^{2}$ value as before. 


In [ ]:
#>>>RUN: L7.4-runcell01

def fnew(x,a,b):
    pVal=b*(1-x)
    return a*x+pVal

model  = lmfit.Model(fnew)
p = model.make_params(a=1000,b=1000)

result = model.fit(data=yhist,x=xhist, params=p, weights=xweights)
lmfit.report_fit(result)
plt.figure()
result.plot()
plt.show()

x0 = np.array([1000,1000])
ps = [x0]
sol=opt.minimize(chi2, x0)
print(sol)

x = np.linspace(sol.x[0]*0.99,sol.x[0]*1.01, 100)
y = np.linspace(sol.x[1]*0.99,sol.x[1]*1.01, 100)
X, Y = np.meshgrid(x, y)
Z = np.array([chi2([x,y]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)
plotColorsAndContours(X,Y,Z)
plt.show()

x = np.linspace(sol.x[0]*0.99,sol.x[0]*1.01, 100)
y = np.linspace(sol.x[1]*0.99,sol.x[1]*1.01, 100)

#Now lets fix one parameter at the minimum, and profile the other
plt.plot(x, chi2([x,sol.x[1]]),label='chi2');
plt.axhline(sol.fun+1, c='red')
plt.xlabel("a-value")
plt.ylabel("$\chi^{2}$")
plt.show()

#Now for the other parameter
plt.plot(y, chi2([sol.x[0],y]),label='chi2');
plt.axhline(sol.fun+1, c='red')
plt.xlabel("b-value")
plt.ylabel("$\chi^{2}$")
plt.show()

Now, critically lets now make the same ellipse plot with our new best fit values for $a$ and $b$, an dour new uncertainties. Additionally, we can also do the 1D uncertainty scans like we did above again, to see how what uncertainties we get on $a$ and $b$. 

In [ ]:
plotColorsAndContours(X,Y,Z)
solX1, solX2 = chi2uncX(sol)
solY1, solY2 = chi2uncY(sol)
sigx=(solX2.root-solX1.root)/2.
sigy=(solY2.root-solY1.root)/2.
plotEllipse(sigx,sigy)
plt.show()


What we can see is that our ellipse differ pretty dramatically from above. Also, we see that our 1D uncertainties are very mis-estimated from what lmfit gives us. The reason for this is that `lmfit` takes into account the full correlation of the variables. Whereas just doing a 1D scan is equivalent to just moving on a single line along the $y$ and $x$ axes. 

<a name='exercises_7_4'></a>     

| [Top](#section_7_0) | [Restart Section](#section_7_4) | [Next Section](#section_7_5) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-7.10.1</span>

Run the above fit with just a regular linear fit given by $f(x) = ax + b$. How does the $\chi^{2}$ change? How do the correlations between the variables change? 


In [ ]:
#>>>EXERCISE
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.



In [ ]:
#>>>SOLUTION

def fnew(x,a,b):
    pVal=b
    return -1*a*x+pVal

model  = lmfit.Model(fnew)
p = model.make_params(a=1000,b=0)
result = model.fit(data=yhist,x=xhist, params=p, weights=xweights)
lmfit.report_fit(result)
plt.figure()
result.plot()
plt.show()

x0 = np.array([-20,1000])
ps = [x0]
sol=opt.minimize(chi2, x0)
print(sol)

x = np.linspace(sol.x[0]*0.5,sol.x[0]*1.5, 100)
y = np.linspace(sol.x[1]*0.99,sol.x[1]*1.01, 100)
X, Y = np.meshgrid(x, y)
Z = np.array([chi2([x,y]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)
plotColorsAndContours(X,Y,Z)
plt.show()


<div style="border:1.5px; border-style:solid; padding: 0.5em; border-color: #90409C; color: #90409C;">

**SOLUTION:**

<pre>
$\chi^{2}$ is to within numerical variations the same. The correlation goes down to almost zero. 

</pre>
        
**EXPLANATION:**
    
By reparametrizing the fit, we ahve allowed the function to be equally as expressive as the above funciton. However, we have changed the roles of $a$ and $b$ in the fit. With this new fit $b$ is in charge of getting the overall scale and $a$ is in charge of getting the trend over $\theta$. Before we had both parameters were in charge of both scale and a combination of slope. Overall our fit has the same shape in the end, but the meaning of $a$ and $b$ is quite different and hence, the off-diagonal terms don't play a big role here. 
    
</div>


<a name='section_7_11'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.11 Correlations Between Fit Parameters: Part 2</h2>     

| [Top](#section_7_0) | [Previous Section](#section_7_10) | [Exercises](#exercises_7_11) |


Now if we go back to our original correlated parameterization, we see that the 1D profile method for obtaining the uncertianties is not the same as the uncertainties we get from the fit.  This is clearly a problem, how can we address this issue? 

<h3>Dealing with Correlated uncertainties</h3>

Looking at our parameters, our uncertainty estimate is smaller than what we observed above. The uncertainties quoted now differ from what we got by varying from the $\chi^{2}$ minimum. What we are doing is moving up and down the 2D plot and looking at $\Delta \chi^{2}$. However, given the parameters are strongly correlated, you see that this doesn't capture the true uncertainty in the sense that we can still move further along $x$ and $y$ and still be within the yellow or even blue ellipse. It's quite clear when you overlay the uncertainty from the quadratic function, which draws circles not ellipses. 

What, then, is the right uncertainty? 

Notice our fit with `lmfit` outputs a parameter $C(a,b)=0.872$. This is in fact the correlation between the parameters of $a$ and $b$. We also see from our optimization function that we get something labelled as the `hess_inv`. This we can write noting the relation of the $\chi^{2}$ Hessian and uncertainties as

$$
\begin{equation}
\chi^{2}(x_{i},\vec{\theta})=\chi^{2}_{min}(x_{i},\vec{\theta})+\frac{1}{2}(\theta_{i}-\theta_{j})^{T}\frac{\partial^{2}}{\partial \theta_{i}\theta_{0}}\chi^{2}_{min}(x_{i},\vec{\theta}_{0})(\theta_{j}-\theta_{0})
\end{equation}
$$

Which allows us to write the uncertainties as

$$
\begin{equation}
 \frac{2}{\sigma^{2}}=\frac{\partial^{2}\chi^{2}}{\partial\theta_{i}\partial\theta_{j}} \\
 \sigma^{2}    = 2\left(\frac{\partial^{2}\chi^{2}}{\partial\theta_{i}\partial\theta_{j}}\right)^{-1} 
\end{equation}
$$

Now in the case where the off diagonals of the Hessian are zero, the parameters were uncorrelated, so 

$$
\begin{equation}
\left(\begin{array}{cc}
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}^{2}} & 0\\
0 & \frac{\partial^{2}\chi^{2}}{\partial\theta_{b}^{2}}
\end{array}\right)\rightarrow\left(\begin{array}{cc}
\frac{2}{\sigma_{a}^{2}} & 0\\
0 & \frac{2}{\sigma_{b}^{2}}
\end{array}\right)
\end{equation}
$$

But here we have something more complicated. However this is a natural way to define correlations. Lets first verify our intuition for our minimization scheme by taking our $2x2$ Hessian metrix and diagonalizing, computing the eigenvectors and the eigenvalues. We can diagonalize the matrix as: 

$$
\begin{equation}
A^{-1}2\left(\begin{array}{cc}
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}^{2}} & \frac{\partial^{2}\chi^{2}}{\partial\theta_{a}\partial\theta_{b}}\\
\frac{\partial^{2}\chi^{2}}{\partial\theta_{a}\partial\theta_{b}} & \frac{\partial^{2}\chi^{2}}{\partial\theta_{b}^{2}}
\end{array}\right)^{-1}A=\left(\begin{array}{cc}
\sigma_{1}^{2} & 0\\
0 & \sigma_{2}^{2}
\end{array}\right)
\end{equation}
$$

It is important to note for any N dimensional Hessian, provided the determinant is not zero, we can find a basis of independent variables that are not correlated. That is to say we can always diagonalize our Hessian, and the eigenvectors of our Hessian are the independent values with variances given by the eigenvalues. These are our true uncertainty values (eigenvalues) and direction of variation (eigenvector).  

When we run our minimizer, we get Hessian inverse, which we can play with. Let's go ahead and look at the Hessian from our minimizer. From the Hessian, we can import numpy's linear algebra toolkit and compute the eigen vectors and values. Finally, we can draw these vectors and values on our 2D scan plot above. Let's do it:


In [ ]:
#>>>RUN: L7.5-runcell01

print(np.sqrt(2*sol.hess_inv))
#The diagonals are the uncertainty lmfit quotes

#Really the best way to do this is to get the eigen values using an linear algebra problem
import numpy.linalg as la
w, v=la.eig(2*sol.hess_inv)
print("values",w,"vectors",v)

#Now lets plot the eigenvectors
from matplotlib.patches import Ellipse
def get_cov_ellipse(cov, centre, nstd, **kwargs):
    """
    Return a matplotlib Ellipse patch representing the covariance matrix
    cov centred at centre and scaled by the factor nstd.

    """
    # Find and sort eigenvalues and eigenvectors into descending order
    eigvals, eigvecs = np.linalg.eigh(cov)
    order = eigvals.argsort()[::-1]
    eigvals, eigvecs = eigvals[order], eigvecs[:, order]

    # The anti-clockwise angle to rotate our ellipse by 
    vx, vy = eigvecs[:,0][0], eigvecs[:,0][1]
    theta = np.arctan2(vy, vx)

    # Width and height of ellipse to draw
    width, height = 2* nstd * np.sqrt(eigvals) #the two here is because its width/height not radius
    return Ellipse(xy=centre, width=width, height=height,angle=np.degrees(theta), **kwargs)

err_ellipse=get_cov_ellipse(2*sol.hess_inv,sol.x,1)
fig, ax = plt.subplots(1, 1)
c = ax.pcolor(X,Y,Z,cmap='RdBu')
fig.colorbar(c, ax=ax)
levels = [0.1,1,2.3,4,9, 16, 25, 36, 49, 64, 81, 100]
for i0 in range(len(levels)):
    levels[i0] = levels[i0]+sol.fun
c = plt.contour(X, Y, Z, levels,colors=['red', 'blue', 'yellow','green'])
ax.add_artist(err_ellipse)
plt.show()

Now the filled in circles make a direct correspondance with the uncertainties above. 

From the above we can now formulate a description of uncertainties in our system. When we had uncorrelated parameters we had a total uncertainty from our $\chi^{2}$ given by 

$$
\begin{equation}
\sigma_{tot}^{2} = \sigma_{a}^2+\sigma_{b}^2
\end{equation}
$$

Now we have the full ellipse

$$
\begin{equation}
\sigma_{tot}^{2} = \sigma_{a}^2+\sigma_{b}^2+2\sigma_{ab}
\end{equation}
$$

Where we define that $\sigma_{ab}=\rm{COV(a,b)}$. There are a number of ways to call this variable, they all are equivalent, but lets be careful to write it out. We can write the error matrix in 2D as:

$$
\begin{equation}
\left(\begin{array}{cc}
\sigma_{a}^{2} & {\rm COV}(a,b)\\
{\rm COV}(a,b) & \sigma_{b}^{2}
\end{array}\right)=\sum_{i=1}^{N}\left(\begin{array}{cc}
\left(a_{i}-\bar{a}\right)^{2} & \left(a_{i}-\bar{a}\right)\left(b_{i}-\bar{b}\right)\\
\left(a_{i}-\bar{a}\right)\left(b_{i}-\bar{b}\right) & \left(b_{i}-\bar{b}\right)^{2}
\end{array}\right)
\end{equation}
$$

where on the right side we have written it in terms of the computation over events. Recall that for a linear regression the slope is just the $\rm{COV(X,Y)/VAR(X)}$, so the covariance matrix is intricately tied with slope. 

We can also write it as the correlation matrix where we normalize by the uncertainties:

$$
\begin{equation}
\rho=\left(\frac{1}{\sigma_{a}}  \frac{1}{\sigma_{b}} \right)^{T}\left(\begin{array}{cc}
1 & \frac{{\rm COV}(a,b)}{\sigma_{a}\sigma_{b}}\\
\frac{{\rm COV}(a,b)}{\sigma_{a}\sigma_{b}} & 1
\end{array}\right)\left(\frac{1}{\sigma_{a}}   \frac{1}{\sigma_{b}} \right) 
\end{equation}
$$

Recall that the covariance is what we originally used to compute the linear slope of the points, this is exactly the same here. In fact, instead of scanning the likelihood analytically we could have sampled the points, and done a linear regression. The resulting slope and line can be related to our eigenvectors. One last thing to mention is that if variables are correlated, we can use the correlation to propagate the uncertainties. 

$$
\begin{equation}
\sigma_{f}^{2} = \left(\frac{\partial f}{\partial x}\right)^2\sigma_{x}^2 + \left(\frac{\partial f}{\partial y}\right)^{2}\sigma_{y}^2+\left(\frac{\partial f}{\partial x}\right)\left(\frac{\partial f}{\partial y}\right)\sigma_{xy}
\end{equation}
$$

Now lets check we can get the corelation matrix.

As a small note our final fitted uncertainty is a little bit larger than the Poisson uncertainty. In fact for all fits there is a rule that our uncertainties on any parameter have to be larger than a certain number. This bound is known as the Cramér-Rao bound. I won't derive it or go into in detail, but the Cramér-Rao bound states.

$$
\begin{equation}
\mathrm{Var}(\theta|\hat{\theta}) \geq \frac{1}{\mathcal{I}(\theta)} \\
\mathcal{I}(\theta) = E_{p(X|\theta)}\left[-\frac{\partial^{2}}{\partial\theta^{2}}\log\left(p\left(x|\theta\right)\right)\right]
\end{equation}
$$

Where $\mathrm{Var}(\theta|\hat{\theta})$ is the variance on our fitted parameter and we call $\mathcal{I}(\theta)$ the [Fisher information](https://en.wikipedia.org/wiki/Fisher_information), which generalizes to the correlation matrix over binomial sampled distributions and, in its simplest form, is equal the inverse of the variance of a binomial distribution, which in turn is the more precise description of a poisson distribution. Hence the fact that our uncertainty is larger than the Poisson uncertainty. While I don't want to go into this more, this result is powerful because it means that there is limit to when you should stop searching for a best fit. 

In [ ]:
#>>>RUN: L7.5-runcell02

#Now lets get the correlation C(a,b) (see below)
w, v=np.linalg.eig(2*sol.hess_inv)
print(2*sol.hess_inv)
print(v)
print("c(a,b)",v[0,1]/v[0,0])
print("A deceptively wrong way to get correlation: since its not normalized",sol.hess_inv[0,1]/sol.hess_inv[0,0])

Finally, we are going to run our fit 

In [ ]:
#>>>RUN: L7.5-runcell03

import lmfit

def fnew(x,a,b):
    pVal=b*(1-x)
    return a*x+pVal

#Randomly sample points in the above range
def maketoy(iy):
    toy=np.array([])
    #go through the y-values and Poisson fluctuate
    for i0 in range(len(iy)):
        pVal = np.random.normal (iy[i0],np.sqrt([iy[i0]]))
        toy = np.append(toy,float(pVal))
    return toy

def fittoy(ibin,iy):
    #generate toy
    toy=maketoy(iy)
    #now fit
    model  = lmfit.Model(fnew)
    p = model.make_params(a=1000,b=10)
    xweights=np.array([])
    #setup poison weight
    for i0 in range(len(toy)):
        xweights = np.append(xweights,1./math.sqrt(toy[i0]))
    result = model.fit(data=toy,x=ibin, params=p, weights=xweights)
    return result.params["a"].value,result.params["b"].value

ntoys=2000
lAs=[]
lBs=[]
for i0 in range(ntoys):
    pA,pB=fittoy(xhist,yhist)
    lAs.append(pA)
    lBs.append(pB)
lAs = np.array(lAs)
lBs = np.array(lBs)
    
err_ellipse=get_cov_ellipse(2*sol.hess_inv,sol.x, 1)
fig, ax = plt.subplots(1, 1)
c = ax.pcolor(X,Y,Z,cmap='RdBu')
fig.colorbar(c, ax=ax)
c = plt.contour(X, Y, Z, levels,colors=['red', 'blue', 'yellow','green'])
ax.add_artist(err_ellipse)
plt.plot(lAs,lBs,c='black',marker='.',linestyle = 'None')
plt.xlabel('a')
plt.ylabel('b')
plt.show()

You can see that if we randomly sample from our distributions, then we get the ellipse variation in $A$ and $B$. The toys have a lot of useful elements. We can use it to get the variation in our parameters.  What we can also do is look at the variance of our parameters and compare it to the Hessian that we can diagonalize to get the fit. 

In [ ]:
#Now lets run a. linear regression
def variance(isamples):
    mean=isamples.mean()
    n=len(isamples)
    tot=0
    for pVal in isamples:
        tot+=(pVal-mean)**2
    return tot/n

def covariance(ixs,iys):
    meanx=ixs.mean()
    meany=iys.mean()
    n=len(ixs)
    tot=0
    for i0 in range(len(ixs)):
        tot+=(ixs[i0]-meanx)*(iys[i0]-meany)
    return tot/n

print("A:",lAs.mean(),"+/-",lAs.std())
print("B:",lBs.mean(),"+/-",lBs.std())
print("Cov:",covariance(lAs,lBs),"A Variance:",variance(lAs),"B Variance:",variance(lBs))
print("Check with Hessian:",2*sol.hess_inv)
print("Cor:",covariance(lAs,lBs)/math.sqrt(variance(lAs)*variance(lBs)),"A Variance:",1.,"B Variance:",1.)

<a name='exercises_7_5'></a>   

| [Top](#section_7_0) | [Restart Section](#section_7_5) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-7.11.1</span>

Repeat the above uncorrelated fit $f(x) = a x + b$, plot the ellipse, compute the uncertainties in $a$ and $b$? Do they correspond with lmfit? 


In [ ]:
#>>>EXERCISE
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def exp_func(x):
    return 0


In [ ]:
#>>>SOLUTION
def fnew(x,a,b):
    pVal=b
    return a*x+pVal

##Copying andn pasting everything from above
model  = lmfit.Model(fnew)
p = model.make_params(a=1000,b=0)
result = model.fit(data=yhist,x=xhist, params=p, weights=xweights,scale_covar=False)
lmfit.report_fit(result)
plt.figure()
result.plot()
plt.show()

x0 = np.array([-20,1000])
ps = [x0]
sol=opt.minimize(chi2, x0)
w, v=la.eig(2*sol.hess_inv)
print("values",w,"vectors",v)
print("matrix a:",np.sqrt(w[0]),"b:",np.sqrt(w[1]))
print("a:",result.params["a"].stderr,"b:",result.params["b"].stderr)

x = np.linspace(sol.x[0]*0.5,sol.x[0]*1.5, 100)
y = np.linspace(sol.x[1]*0.99,sol.x[1]*1.01, 100)
X, Y = np.meshgrid(x, y)
Z = np.array([chi2([x,y]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)
err_ellipse=get_cov_ellipse(2*sol.hess_inv,sol.x,1)
fig, ax = plt.subplots(1, 1)
c = ax.pcolor(X,Y,Z,cmap='RdBu')
fig.colorbar(c, ax=ax)
levels = [0.1,1.0,2.3,4,9, 16, 25, 36, 49, 64, 81, 100]
for i0 in range(len(levels)):
    levels[i0] = levels[i0]+sol.fun
c = plt.contour(X, Y, Z, levels,colors=['red', 'blue', 'yellow','green'])
ax.add_artist(err_ellipse)
plt.show()

<div style="border:1.5px; border-style:solid; padding: 0.5em; border-color: #90409C; color: #90409C;">

**SOLUTION:**

<pre>
We find the uncertainties are comparable to lmfit. 
</pre>
        
**EXPLANATION:**
    
You will notice that to get the uncertainties we used this `scale_covar=False` formula, this computes the uncertainties from the Hessian in lmfit. Otherwise, it applies a correction to the uncertainties to account for the scenario when the fit $\chi^{2}$ is large. 
    
</div>


<a name='section_7_12'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L8.1 Supernovae and the universe in 2D</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_11) | [Exercises](#exercises_7_12) |


Now, we would like to end our 2D plotting discussion and move on to geener pastures by doing the full supernovae analysis and making the famous 2D plot of dark matter density. 

Let's go ahead and load the supernova data once more time. 

In [ ]:
import math
import numpy as np
import csv
import matplotlib.pyplot as plt
from scipy import stats

#Let's try to understand how good the fits we made in last Lesson are, let's load the supernova data again
label='data/sn_z_mu_dmu_plow_union2.1.txt'

def distanceconv(iMu):
    power=iMu/5+1
    return 10**power

def distanceconverr(iMu,iMuErr):
    power=iMu/5+1
    const=math.log(10)/5.
    return const*(10**power)*iMuErr

def load(iLabel,iMaxZ=10.):
    redshift=np.array([])
    distance=np.array([])
    distance_err=np.array([])
    with open(iLabel,'r') as csvfile:
        plots = csv.reader(csvfile, delimiter='\t')
        for row in plots:
            if float(row[1]) > iMaxZ:
                continue
            redshift = np.append(redshift,float(row[1]))
            distance = np.append(distance,distanceconv(float(row[2])))
            distance_err = np.append(distance_err,distanceconverr(float(row[2]),float(row[3])))
    return redshift,distance,distance_err  
        
#Now let's run the regression again
def variance(isamples):
    mean=isamples.mean()
    n=len(isamples)
    tot=0
    for pVal in isamples:
        tot+=(pVal-mean)**2
    return tot/n

def covariance(ixs,iys):
    meanx=ixs.mean()
    meany=iys.mean()
    n=len(ixs)
    tot=0
    for i0 in range(len(ixs)):
        tot+=(ixs[i0]-meanx)*(iys[i0]-meany)
    return tot/n

def linear(ix,ia,ib):
    return ia*ix+ib

redshift,distance,distance_err=load(label)
var=variance(redshift)
cov=covariance(redshift,distance)
A=cov/var
const=distance.mean()-A*redshift.mean()
xvals = np.linspace(0,1.5,100)
yvals = []
for pX in xvals:
    yvals.append(linear(pX,A,const))

plt.plot(xvals,yvals)
plt.errorbar(redshift,distance,yerr=distance_err,marker='.',linestyle = 'None', color = 'black')
plt.xlabel("z(redshift)")
plt.ylabel("distance(pc)")
plt.show()

<h3>Remember Numerical properties of the universe</h3>

Recall in Lecture 6 we took the following formula for the comoving distance of the redshift

$$ d(z) = \frac{c}{h_{0}}(1+z) \sum_{i=0}^{i=100} \frac{dz}{\sqrt{\Omega_{M}(1+z_{i}^{\prime})^{3} + 1-\Omega_{M}}}$$

where $dz=z/100$ and $z_{i}= dz \times i$. 

Then we went ahead and coded this up and put fit it to our supernova data. Let's run the fit and look at our previous diagnostics. 



In [ ]:
#>>>RUN: L6.8-runcell01
import lmfit

def hubble(z,Om):
    pVal=Om*(1+z)**3+(1.-Om)
    return np.sqrt(pVal)

def lumidistance(z,h0,Om):
    integral=0
    nint=100
    for i0 in range(nint):
        zp=z*float(i0)/100.
        dz=z/float(nint)
        pVal=1./(1e-5+hubble(zp,Om))
        integral += pVal*dz
    d=(1.+z)*integral*(1e6*3e5/h0)
    return d

model  = lmfit.Model(lumidistance)
p = model.make_params(h0=70,Om=0.2)
result = model.fit(data=distance, params=p, z=redshift, weights=1./distance_err)
lmfit.report_fit(result)

#Plot it against the data
xvals = np.linspace(0,1.4,100)
yvals = []
for pX in xvals:
    yvals.append(lumidistance(pX,result.params["h0"],result.params["Om"]))

plt.errorbar(redshift,distance,yerr=distance_err,marker='.',linestyle = 'None', color = 'black')
plt.plot(xvals,yvals)
plt.show()

x = np.linspace(len(distance)*0.5,len(distance)*1.5)
chi2d=stats.chi2.pdf(x,len(distance-2)) # 40 bins
plt.plot(x,chi2d,label='chi2')
plt.axvline(result.chisqr, c='red')
plt.legend(loc='lower right')
plt.show()



Ok, the fit looks really good. Now that we have done this, lets go ahead and extend this to a 2-dimensional plot, to do the 2D plot, we now need to simulatneously scan both the two varaibles that we are fitting for, which in this is Hubble's constant $h_{0}$ and the matter density $\Omega_{m}$. 

To do this, we will run the minimizer and then we will look at the contours about the minimum. 

In [ ]:
#Plot it against the data
def loglike(x):
    lTot=0
    for i0 in range(len(redshift)):
        xtest=lumidistance(redshift[i0],x[0],x[1])
        #lTot = lTot+(distance[i0]-xtest)**2
        lTot = lTot+((1./distance_err[i0])**2)*(distance[i0]-xtest)**2
    return lTot #*0.5 The above is 2 times loglike

#x0 = np.array([70.,0.3])
#ps = [x0]
#bnds = ((0, 1000), (0, 1.5))
#sol=opt.minimize(loglike, x0,bounds=bnds, tol=1e-6)
#sol=opt.minimize(loglike, x0,bounds=bnds)

#Finally lets get our 1D unctainies from the Hessian
#unc=np.sqrt(2*sol.hess_inv.matmat(np.eye(2)))
#print("Unc matrix:",unc)
#print("h0",sol.x[0],"+/-",unc[0,0])
#print("Om:",sol.x[1],"+/-",unc[1,1])

#And lets get the correlations
#import numpy.linalg as la
#w, v=la.eig(2*sol.hess_inv)
#print("values",w,"vectors",v)
#Now lets get the correlation C(a,b) (see below)
#print("c(a,b)",v[0,1]/v[0,0])

#Now lets scan the parameters
x = np.linspace(result.params['h0']*0.9,result.params['h0']*1.1, 30)
y = np.linspace(0.,0.5, 30)
X, Y = np.meshgrid(x, y)
def pvalue(isigma):
    return stats.norm.cdf(isigma)-stats.norm.cdf(-isigma)

OneSigma   = stats.chi2.ppf(pvalue(1),2)
TwoSigma   = stats.chi2.ppf(pvalue(2),2)
ThreeSigma = stats.chi2.ppf(pvalue(3),2)
print("Levels:",OneSigma,TwoSigma,ThreeSigma)
levels = [OneSigma,TwoSigma,ThreeSigma]
for i0 in range(len(levels)):
    levels[i0] = levels[i0]+loglike([result.params['h0'],result.params['Om']])

Z = np.array([loglike([x,y]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)

fig, ax = plt.subplots(1, 1)
c = ax.pcolor(X,Y,Z,cmap='RdBu')
fig.colorbar(c, ax=ax)
c = plt.contour(X, Y, Z, levels,colors=['red', 'blue', 'yellow','green'])
plt.xlabel('h0')
plt.ylabel('$\Omega_{M}$')
plt.show()



<h3>Fitting for the curvature</h3>


So given the Friedmann equations, we can add back the curvature term:

$$ 
d(z) = ct^{\prime} = (1+z)ct = (1+z)\frac{c}{h_{0}}\int_{0}^{z} \frac{dz^{\prime}}{\sqrt{\Omega_{M}\left(1+z^{\prime}\right)^{3} + \Omega_{\kappa}\left(1+z^{\prime}\right)^{2}+ 1-\Omega_{M}-\Omega_{\kappa}}}
$$

We can adjust the luminosity distance to add this back. Recall that 

$$
\Omega_{m}+\Omega_{k}+\Omega_{\Lambda}=1
$$
So we only need to float two fo them. Let's go ahead and run the fit. 

$$ 
d(z) = ct^{\prime} = (1+z)ct = (1+z)\frac{c}{h_{0}}\int_{0}^{z} \frac{dz^{\prime}}{\sqrt{\Omega_{M}\left(1+z^{\prime}\right)^{3} + (1-\Omega_{\Lambda}-\Omega_{M})\left(1+z^{\prime}\right)^{2}+ \Omega_{\Lambda}}}
$$


In [ ]:
#And finally lets actually float both the matter and dark energy
def sqrtTerm2(z,Om,OmL):
    pVal=Om*(1+z)**3+OmL+(1-OmL-Om)*(1+z)**2
    return np.sqrt(pVal)

def lumidistance2(x,h0,Om,OmL):
    integral=0
    nint=100
    for i0 in range(nint):
        pVal=1./(1e-5+sqrtTerm2(x*float(i0)/100.,Om,OmL))
        integral += pVal*x/float(nint)
    d=(1.+x)*integral*(1e6*3e5/h0)
    return d

def loglike2(x):
    lTot=0
    for i0 in range(len(redshift)):
        xtest=lumidistance2(redshift[i0],x[0],x[1],x[2])
        #lTot = lTot+(distance[i0]-xtest)**2
        lTot = lTot+((1./distance_err[i0])**2)*(distance[i0]-xtest)**2
    return lTot #*0.5 The above is 2 times loglike


model  = lmfit.Model(lumidistance2)
p = model.make_params(h0=70,Om=0.3,OmL=0.7)
result = model.fit(data=distance, params=p, x=redshift, weights=1./distance_err)
lmfit.report_fit(result)
#plt.figure()
#result.plot()

#Plot it against the data
xvals = np.linspace(0,1.4,100)
yvals = []
for pX in xvals:
    yvals.append(lumidistance2(pX,result.params["h0"],result.params["Om"],result.params["OmL"]))

plt.errorbar(redshift,distance,yerr=distance_err,marker='.',linestyle = 'None', color = 'black')
plt.plot(xvals,yvals)
plt.show()


Now lets see if we can make our plot. We will draw the log likelihood about the best fit from lmfit, and then we will draw the contours. Also lets not go overboard on the contours

In [ ]:
#result.params["a"].value,result.params["b"].value
#Now lets scan the parameters
x = np.linspace(0,1.5,30)
y = np.linspace(0,1.5,30)
X, Y = np.meshgrid(x, y)
levels = [OneSigma,TwoSigma,ThreeSigma]
for i0 in range(len(levels)):
    levels[i0] = levels[i0]+loglike2([result.params["h0"],result.params["Om"],result.params["OmL"]])
Z = np.array([loglike2([result.params["h0"],x,y]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)
fig, ax = plt.subplots(1, 1)
c = ax.pcolor(X,Y,Z,cmap='RdBu')
fig.colorbar(c, ax=ax)
c = plt.contour(X, Y, Z, levels,colors=['red', 'blue', 'yellow','green'])
plt.xlabel('$\Omega_{m}$')
plt.ylabel('$\Omega_{\Lambda}$')
plt.show()

This finally gives us our 2D plot that, and you can see that it looks good

<a name='section_8_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L8.4 Principal Component Analysis</h2>     

| [Top](#section_8_0) | [Previous Section](#section_8_3) | [Exercises](#exercises_8_4) |


<h3>Overview</h3>

Finally, I would like to say that this method of finding the ellipse is our first deep learning method.
This procedure of computing the covariance matrix, and finding the eigenvectors is known as  principal component analysis or PCA. Let's run it on our example and look from <a href="https://jakevdp.github.io/PythonDataScienceHandbook/05.09-principal-component-analysis.html" target="_blank">Python Data Science Handbook by Jake VanderPlas</a>.

First, what we can do is look at our old correlated fit. All this will do is get the eigenvectors and values for our 2D plot.

In [ ]:
#>>>RUN: L8.4-runcell01
import numpy as np
from sklearn.decomposition import PCA
import numpy.linalg as la
#make some toy data
lAs = np.random.normal(0,1,1000)
lBs = np.random.normal(0,1,1000)+0.5*lAs
#print(lAs)

cov = np.cov([lAs,lBs])
#eigen cov
w, v=la.eig(cov)


plt.plot(lAs,lBs,".")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

X=(np.vstack([lAs,lBs])).T
pca = PCA(n_components=2)
pca.fit(X)
print("PCA vectors")
print(pca.components_)
print("PCA values")
print(pca.explained_variance_)
print("Old Eigen","vectors",w,"values",v)

def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',linewidth=2,shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

# plot data
plt.scatter(lAs, lBs, alpha=0.2)
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)
plt.axis('equal');

Finally, we can try this on an ML dataset. Let's take images with many pixels and treat each pixel as a separate dimension. We can then run the decomposition on the image by decomposing the n-pixel by n-pixel correlation matrix. What we will do is take an image that is 62x47=2914 Pixels. From that we can compute the covariance matrix and we can start to diagonlize this. Since this is a large matrix (2914x2914), we have to use sophisticaed eigen decomposition strategies. In this case, we will use something called RandomizedPCA. 

What we are going to do is run PCA on the images, and take the top 200 eigenvectors from that. Then we are going to plot the top few of these eigen vectors and the cumulative explained variance, which is a metric for how much information can be gained by adding dimensions. 

In [ ]:
#>>>RUN: L8.4-runcell02

#Now let's do it ML style for fun
#Load some faces of images
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person=60)

fig, axes = plt.subplots(3, 8, figsize=(9, 4),subplot_kw={'xticks':[], 'yticks':[]},gridspec_kw=dict(hspace=0.1, wspace=0.1))
#Let's plot the eigenvectors
for i, ax in enumerate(axes.flat):
    ax.imshow(faces.data[i].reshape(62, 47), cmap='bone')
#print(62*47)
#a
#Fit them to PCA 
from sklearn.decomposition import PCA as RandomizedPCA
pca = RandomizedPCA(200)
pca.fit(faces.data)
fig, axes = plt.subplots(3, 8, figsize=(9, 4),subplot_kw={'xticks':[], 'yticks':[]},gridspec_kw=dict(hspace=0.1, wspace=0.1))
#Let's plot the eigenvectors
for i, ax in enumerate(axes.flat):
    ax.imshow(pca.components_[i].reshape(62, 47), cmap='bone')
plt.show()
    
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');
plt.show()


What you can see above is that the top eigen vectors capture the shape of the face. You can see the shape variations between the different eigenvectors as you go through. Youc an see also that about 80% of the total info in the first 200 eigenvectors is captures in the first 25 components. 

Finally, what we can do is plot our world leaders just by taking the first 80 eigenvectors of our sample. What we have effectively done is compress our original image into just 80 values, thats all!

In [ ]:
#>>>RUN: L8.4-runcell03

# Compute the components and projected faces
pca = RandomizedPCA(10).fit(faces.data)
components = pca.transform(faces.data)
projected = pca.inverse_transform(components)

# Plot the results
fig, ax = plt.subplots(2, 10, figsize=(10, 2.5),subplot_kw={'xticks':[], 'yticks':[]},gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i in range(10):
    ax[0, i].imshow(faces.data[i].reshape(62, 47), cmap='binary_r')
    ax[1, i].imshow(projected[i].reshape(62, 47), cmap='binary_r')
    
ax[0, 0].set_ylabel('full-dim\ninput')
ax[1, 0].set_ylabel('80-dim\nreconstruction');


<h3>Exercise using stellar spectra</h3>

As a final exercise, we are going to isolate features from stellar spectra of the Sloan Digit Sky Survey. To that, let's first load data from the SDSS and try to understand what this looks like. We will use the `astroML` package, that allows us to pull astro data quickly. 

From this data, we will look at redshift correct galaxy spectra. The correction allows us to look at the large population of galaxies in this dataset and try to find the dominant features. Let's take a quick look at the data. 

In [ ]:
#>>>RUN: L8.4-runcell04

from astroML.datasets import sdss_corrected_spectra

data = sdss_corrected_spectra.fetch_sdss_corrected_spectra()
wavelengths = sdss_corrected_spectra.compute_wavelengths(data)
spectra_raw = data['spectra']
count=-1
#let's plot this guy
fig, ax = plt.subplots(2, 10, figsize=(20, 5.5),subplot_kw={'yticks':[]},gridspec_kw=dict(hspace=0.1, wspace=0.1))
nrows = 2; ncols = 10
for i in range(ncols):
    for j in range(nrows):
        count=count+1
        ax[j,i].plot(wavelengths,spectra_raw[count], '-k', lw=1)
        ax[j,i].set_xlim(3100, 7999)
        if j < nrows - 1:
            ax[j,i].xaxis.set_major_formatter(plt.NullFormatter())
        else:
            ax[j,i].set_xlabel(r'wavelength $(\AA)$')
            
plt.show()


What you see is that the spectra vary, but there are a few lines from the galaxies that are particularly bright. Look at the one above 6000 Angstroms. That the Hydrogen, $\beta$ emission line. You can see the whole table of lines <a href="http://astronomy.nmsu.edu/drewski/tableofemissionlines.html" target="_blank">here</a>.

In the following problem, we will look closely at the dominant eigenvector and identify any spectral lines that it might include.

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-8.4.1</span>

Your task is run PCA, look at the dominant eigenvector, and find the top two spectral lines. What do they correspond to (refer to the table <a href="http://astronomy.nmsu.edu/drewski/tableofemissionlines.html" target="_blank">here</a>.)?

Select two spectral lines from the options below (in units of angstroms):

- He ~4200
- H$\gamma$ ~4350
- H$\beta$ ~4860
- O III ~4950
- O III ~5006
- H$\alpha$ ~6560
- N II ~6580
- O I ~7000

Does this make sense?


In [ ]:
#>>>EXERCISE: L8.4.1

# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.
from astroML.datasets import sdss_corrected_spectra

data = sdss_corrected_spectra.fetch_sdss_corrected_spectra()
wavelengths = sdss_corrected_spectra.compute_wavelengths(data)
spectra_raw = data['spectra']

#select only the first 850 bins to avoid noise features
spectra_raw = spectra_raw[:,0:850]
wavelengths = wavelengths[0:850]
pca = RandomizedPCA()


#YOUR CODE HERE
#fit the pca to the spectra_raw data 
#plot the first eigenvector, pca.components_[0] over the full range of wavelengths
#optionally constrain the wavelength range to zoom in on spectral lines

In [ ]:
#>>>SOLUTION: L8.4.1

from astroML.datasets import sdss_corrected_spectra

data = sdss_corrected_spectra.fetch_sdss_corrected_spectra()
wavelengths = sdss_corrected_spectra.compute_wavelengths(data)
spectra_raw = data['spectra']
spectra_raw = spectra_raw[:,0:850]
wavelengths = wavelengths[0:850]

pca = RandomizedPCA()
pca.fit(spectra_raw)
plt.plot(wavelengths,pca.components_[0] , '-k', lw=1)
plt.xlabel(r'wavelength $(\AA)$')
#plt.xlim(6500,7000)
plt.show()

#print(wavelengths,pca.components_)
#print(pca.explained_variance_ratio_)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');
plt.show()


<div style="border:1.5px; border-style:solid; padding: 0.5em; border-color: #90409C; color: #90409C;">

**SOLUTION:**

<pre>

- O III ~5006
- H$\alpha$ ~6560

</pre>
        
**EXPLANATION:**
    
These emissions lines are famous lines from Oxygen and Hydrogen. See for example <a href="https://en.wikipedia.org/wiki/Doubly_ionized_oxygen" target="_blank">here</a>.
    
</div>


>#### Follow-up 8.4.1a (ungraded)
>
>Plot all of the eigenvectors. What are the dominant features of each?

In [ ]:
#>>>SOLUTION: L8.4.1

#Now let's look at the top 100 eigen vectors 
count=-1
fig = plt.figure(figsize=(20, 20))
fig.subplots_adjust(left=0.05, right=0.95, wspace=0.05,bottom=0.1, top=0.95, hspace=0.05)
nrows = 10; ncols = 10
for i in range(ncols):
    for j in range(nrows):
        count=count+1
        ax = fig.add_subplot(nrows, ncols, ncols * j + 1 + i)
        ax.plot(wavelengths,pca.components_[count] , '-k', lw=1)
        ax.set_xlim(3100, 7999)

        ax.yaxis.set_major_formatter(plt.NullFormatter())
        ax.xaxis.set_major_locator(plt.MultipleLocator(1000))
        if j < nrows - 1:
            ax.xaxis.set_major_formatter(plt.NullFormatter())
        else:
            plt.xlabel(r'wavelength $(\AA)$')
plt.show()

In [ ]:
components = pca.transform(spectra_raw)
projected = pca.inverse_transform(components)

# Plot the results
fig, ax = plt.subplots(2, 5, figsize=(20, 10),subplot_kw={'xticks':[], 'yticks':[]},gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i in range(5):
    ax[0,i].plot(wavelengths,spectra_raw[count], '-k', lw=1)
    ax[0,i].set_xlim(3100, 7999)
    ax[1,i].plot(wavelengths,projected[count], '-k', lw=1)
    ax[1,i].set_xlim(3100, 7999)
    
ax[0, 0].set_ylabel('full-dim\ninput')
ax[1, 0].set_ylabel('80-dim\nreconstruction');
